# Fourier Flows (Casseau Ch. 3.2.1)

This notebook reproduces the first two Fourier-flow verification cases from Casseau (pure N2 and homogeneous air).
We use the current diffusion model and the new Maxwell-slip / Smoluchowski-jump wall BC.

Notes:
- Case 1: pure N2 (Knov = 0.1).
- Case 2a: homogeneous air (0.5 N2 / 0.5 O2), Knov = 0.002.
- Case 2b: homogeneous air, Knov = 0.1.

You can paste digitized Casseau datapoints into the reference arrays below to overlay in the plots.


In [ ]:
from __future__ import annotations

from pathlib import Path

import jax.numpy as jnp
import numpy as np
import matplotlib.pyplot as plt

from compressible_core import (
    chemistry_utils,
    energy_models,
    transport as transport_core,
    transport_casseau,
)
from compressible_2d import mesh_gmsh
from compressible_2d import equation_manager
from compressible_2d import equation_manager_types
from compressible_2d import equation_manager_utils
from compressible_2d import numerics_types


def find_repo_root() -> Path:
    cwd = Path.cwd().resolve()
    for parent in [cwd, *cwd.parents]:
        if (parent / "pyproject.toml").exists():
            return parent
    return cwd

In [ ]:
def build_channel_mesh(nx: int, ny: int, Lx: float, H: float) -> mesh_gmsh.Mesh2D:
    """Structured quad mesh for a 2D channel."""
    x = np.linspace(0.0, Lx, nx + 1)
    y = np.linspace(0.0, H, ny + 1)
    nodes = []
    for j in range(ny + 1):
        for i in range(nx + 1):
            nodes.append([x[i], y[j]])
    nodes = np.asarray(nodes, dtype=float)

    def node_id(i, j):
        return j * (nx + 1) + i

    cells = []
    for j in range(ny):
        for i in range(nx):
            n0 = node_id(i, j)
            n1 = node_id(i + 1, j)
            n2 = node_id(i + 1, j + 1)
            n3 = node_id(i, j + 1)
            cells.append([n0, n1, n2, n3])

    # Boundary tags
    TAG_LEFT = 1
    TAG_RIGHT = 2
    TAG_BOTTOM = 3
    TAG_TOP = 4

    boundary_edges = []
    # Left and right
    for j in range(ny):
        boundary_edges.append((node_id(0, j), node_id(0, j + 1), TAG_LEFT))
        boundary_edges.append((node_id(nx, j), node_id(nx, j + 1), TAG_RIGHT))
    # Bottom and top
    for i in range(nx):
        boundary_edges.append((node_id(i, 0), node_id(i + 1, 0), TAG_BOTTOM))
        boundary_edges.append((node_id(i, ny), node_id(i + 1, ny), TAG_TOP))

    return mesh_gmsh.Mesh2D.from_cells(nodes, cells, boundary_edges)

In [ ]:
def load_species_table(species_names: tuple[str, ...]) -> chemistry_utils.SpeciesTable:
    repo_root = find_repo_root()
    data_dir = repo_root / "data"
    general_data = data_dir / "species.json"
    bird_data = data_dir / "air_5_bird_energy.json"

    energy_cfg = energy_models.EnergyModelConfig(
        model="bird",
        include_electronic=False,
        data_path=str(bird_data),
    )

    return chemistry_utils.load_species_table(
        species_names=species_names,
        general_data_path=str(general_data),
        energy_model_config=energy_cfg,
    )

In [ ]:
def number_density_to_rho_Y(species, n_by_species):
    """Convert number densities (1/m^3) to rho and mass fractions."""
    m_s = species.molar_masses / 6.02214076e23  # kg per particle
    n_by_species = np.asarray(n_by_species, dtype=float)
    rho_s = n_by_species * m_s
    rho = np.sum(rho_s)
    Y = rho_s / rho
    return rho, Y

In [ ]:
def make_equation_manager(species, collision_integrals, H, U, Tw_bottom, Tw_top):
    TAG_LEFT = 1
    TAG_RIGHT = 2
    TAG_BOTTOM = 3
    TAG_TOP = 4

    repo_root = find_repo_root()
    data_dir = repo_root / "data"
    casseau_transport = transport_casseau.load_casseau_transport_table(
        data_dir / "air_5_casseau_transport.json", species.names
    )

    numerics_config = numerics_types.NumericsConfig2D(
        dt=1e-5,
        cfl=0.4,
        dt_mode="fixed",
        integrator_scheme="rk2",
        spatial_scheme="first_order",
        flux_scheme="hllc",
        axisymmetric=False,
        clipping=numerics_types.ClippingConfig2D(),
    )

    boundary_config = equation_manager_types.BoundaryConditionConfig2D(
        tag_to_bc={
            TAG_LEFT: {"type": "outflow"},
            TAG_RIGHT: {"type": "outflow"},
            TAG_BOTTOM: {
                "type": "wall_slip",
                "Tw": Tw_bottom,
                "u_wall": U,
                "v_wall": 0.0,
                "sigma_t": 1.0,
                "sigma_v": 1.0,
            },
            TAG_TOP: {
                "type": "wall_slip",
                "Tw": Tw_top,
                "u_wall": U,
                "v_wall": 0.0,
                "sigma_t": 1.0,
                "sigma_v": 1.0,
            },
        }
    )

    eq_manager = equation_manager_types.EquationManager2D(
        species=species,
        collision_integrals=collision_integrals,
        reactions=None,
        numerics_config=numerics_config,
        boundary_config=boundary_config,
        transport_model=equation_manager_types.TransportModelConfig(
            model="casseau", include_diffusion=True
        ),
        casseau_transport=casseau_transport,
    )
    return eq_manager

In [ ]:
def run_case(
    species_names,
    n_by_species,
    case_name,
    nx=6,
    ny=50,
    Lx=1.0,
    H=1.0,
    U=300.0,
    Tw_bottom=2000.0,
    Tw_top=3000.0,
    T_init=2500.0,
    t_final=5e-3,
):
    species = load_species_table(species_names)

    repo_root = find_repo_root()
    ci_path = repo_root / "data" / "collision_integrals_tp2867.json"
    collision_integrals = transport_core.create_collision_integral_table_from_json(
        ci_path
    )

    eq_manager = make_equation_manager(
        species, collision_integrals, H, U, Tw_bottom, Tw_top
    )

    mesh = build_channel_mesh(nx, ny, Lx, H)

    rho, Y = number_density_to_rho_Y(species, n_by_species)
    n_cells = mesh.cell_areas.shape[0]
    Y_field = np.broadcast_to(Y[None, :], (n_cells, len(Y)))

    rho_field = np.full((n_cells,), rho)
    u_field = np.full((n_cells,), U)
    v_field = np.zeros((n_cells,))
    T_field = np.full((n_cells,), T_init)
    Tv_field = np.full((n_cells,), T_init)

    U_init = equation_manager_utils.compute_U_from_primitives(
        Y_s=jnp.asarray(Y_field),
        rho=jnp.asarray(rho_field),
        u=jnp.asarray(u_field),
        v=jnp.asarray(v_field),
        T_tr=jnp.asarray(T_field),
        T_V=jnp.asarray(Tv_field),
        equation_manager=eq_manager,
    )

    U_hist, t_hist = equation_manager.run_scan(
        U_init, mesh, eq_manager, t_final=t_final, save_interval=10
    )
    return {
        "name": case_name,
        "mesh": mesh,
        "eq_manager": eq_manager,
        "U_hist": U_hist,
        "t_hist": t_hist,
        "Tw_bottom": Tw_bottom,
    }

In [ ]:
# Case definitions (number densities in 1/m^3)
# Values from Casseau Table 3.3, units are 1e19 m^-3
case1_n = [2.086e19]  # N2
case2a_n = [52.15e19, 52.15e19]  # N2, O2
case2b_n = [1.043e19, 1.043e19]  # N2, O2

cases = []
cases.append(run_case(("N2",), case1_n, "case1", t_final=5e-3))
cases.append(run_case(("N2", "O2"), case2a_n, "case2a", t_final=5e-3))
cases.append(run_case(("N2", "O2"), case2b_n, "case2b", t_final=5e-3))

In [ ]:
def extract_profile(case, x_target=None):
    mesh = case["mesh"]
    eq_manager = case["eq_manager"]
    U = case["U_hist"][-1]

    prim = equation_manager_utils.extract_primitives(U, eq_manager)
    x = mesh.cell_centroids[:, 0]
    y = mesh.cell_centroids[:, 1]

    if x_target is None:
        x_target = 0.5 * (x.min() + x.max())
    dx = np.min(np.diff(np.unique(x))) if len(np.unique(x)) > 1 else 1.0
    mask = np.abs(x - x_target) <= 0.25 * dx

    y_sel = y[mask]
    T_sel = np.asarray(prim.T)[mask]
    Tv_sel = np.asarray(prim.Tv)[mask]

    order = np.argsort(y_sel)
    y_sorted = y_sel[order]
    T_sorted = T_sel[order]
    Tv_sorted = Tv_sel[order]

    y_over_H = y_sorted / (y.max() - y.min())
    Tb0 = case["Tw_bottom"]
    T_norm = T_sorted / Tb0
    Tv_norm = Tv_sorted / Tb0

    return y_over_H, T_norm, Tv_norm

In [ ]:
# Reference data placeholders (fill with digitized Casseau points)
ref_case1 = {
    "y_over_H": np.array([]),
    "Ttr_over_Tb0": np.array([]),
    "Tv_over_Tb0": np.array([]),
}
ref_case2a = {
    "y_over_H": np.array([]),
    "Ttr_over_Tb0": np.array([]),
    "Tv_over_Tb0": np.array([]),
}
ref_case2b = {
    "y_over_H": np.array([]),
    "Ttr_over_Tb0": np.array([]),
    "Tv_over_Tb0": np.array([]),
}

In [ ]:
profiles = {}
for case in cases:
    profiles[case["name"]] = extract_profile(case)

fig, axes = plt.subplots(1, 2, figsize=(10, 4), sharey=True)

# T_tr profiles
ax = axes[0]
for name, (y, Tn, _Tv) in profiles.items():
    ax.plot(Tn, y, label=f"{name}")
if ref_case1["y_over_H"].size:
    ax.scatter(
        ref_case1["Ttr_over_Tb0"], ref_case1["y_over_H"], s=12, label="Casseau case1"
    )
if ref_case2a["y_over_H"].size:
    ax.scatter(
        ref_case2a["Ttr_over_Tb0"], ref_case2a["y_over_H"], s=12, label="Casseau case2a"
    )
if ref_case2b["y_over_H"].size:
    ax.scatter(
        ref_case2b["Ttr_over_Tb0"], ref_case2b["y_over_H"], s=12, label="Casseau case2b"
    )
ax.set_xlabel("T_tr / T_b0")
ax.set_ylabel("y / H")
ax.set_title("Normalized T_tr")
ax.legend()

# T_v profiles
ax = axes[1]
for name, (y, _Tn, Tv) in profiles.items():
    ax.plot(Tv, y, label=f"{name}")
if ref_case1["y_over_H"].size:
    ax.scatter(
        ref_case1["Tv_over_Tb0"], ref_case1["y_over_H"], s=12, label="Casseau case1"
    )
if ref_case2a["y_over_H"].size:
    ax.scatter(
        ref_case2a["Tv_over_Tb0"], ref_case2a["y_over_H"], s=12, label="Casseau case2a"
    )
if ref_case2b["y_over_H"].size:
    ax.scatter(
        ref_case2b["Tv_over_Tb0"], ref_case2b["y_over_H"], s=12, label="Casseau case2b"
    )
ax.set_xlabel("T_v / T_b0")
ax.set_title("Normalized T_v")
ax.legend()

plt.tight_layout()
plt.show()